In [2]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from csv import writer
import re
from selenium import webdriver
import time
import requests
import numpy as np

In [3]:
df_bali = pd.read_csv('Bali-clean.csv', dtype=str)
df_bali_r = pd.read_csv('Bali-clean-r.csv', dtype=str)

In [3]:
df_bali

,kabupaten,kecamatan,desa
0,Jembrana,Melaya,Gilimanuk
1,Jembrana,Melaya,Melaya
2,Jembrana,Melaya,Candikusuma
3,Jembrana,Melaya,Tuwed
4,Jembrana,Melaya,Tukadaya
...,...,...,...
711,Buleleng,Tejakula,Tejakula
712,Buleleng,Tejakula,Les
713,Buleleng,Tejakula,Penuktukan
714,Buleleng,Tejakula,Sambirenteng


# Iterasi per desa

In [4]:
no_desa = 569
teks_pat = f'{df_bali_r["desa"][no_desa]}--{df_bali_r["kecamatan"][no_desa]}--{df_bali_r["kabupaten"][no_desa]}'
print(teks_pat)
df_all = pd.DataFrame(columns = ['link'])

Pejarakan--Gerokgak--Buleleng


In [9]:
teks = f'https://www.balipost.com/peristiwa'
html = urlopen(teks)
soup = BeautifulSoup(html, 'lxml')
time.sleep(2)
# get title
print(soup.title)
all_links = soup.find_all("a")
list_address = list()
for link in all_links:
    link = link.get("href")
    list_address.append(''+link)

<title>Peristiwa Arsip | BALIPOST.com</title>


In [12]:
# import requests
# from bs4 import BeautifulSoup

# size = 24
# page = 0

# hasNext = True
# while hasNext == True:
#     page +=1
#     print('\tPage: %s' %page)
#     url = 'https://www.balipost.com/peristiwa'.format(size=size, page=page)
#     jsonData = requests.get(url).json()
    
#     hasNext = jsonData['hasNext']

#     soup = BeautifulSoup(jsonData['html'], 'html.parser')
#     cardTitles = soup.find_all('a',{'class':'recipeCard__title'})
#     for title in cardTitles:
#         print(title.text.strip())

In [6]:
# teks = f'https://www.airbnb.co.id/s/{df_bali["desa"][no_desa]}--{df_bali["kecamatan"][no_desa]}--{df_bali["kabupaten"][no_desa]}/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&date_picker_type=calendar&l2_property_type_ids%5B%5D=4&search_type=search_query&source=structured_search_input_header'
# teks = 'https://www.airbnb.co.id/s/Gilimanuk--Melaya--Jembrana/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&date_picker_type=calendar&l2_property_type_ids%5B%5D=4&search_type=search_query&source=structured_search_input_header&query=Gilimanuk%2C%20Melaya%2C%20Jembrana%2C%20Kabupaten%20Jembrana%2C%20Bali%2C%20Indonesia&place_id=ChIJ8fIsf4ZD0S0R_n8kIVlO9iA&federated_search_session_id=8f660834-8bff-43c7-a976-78e6221856f1&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoyMTYsInZlcnNpb24iOjF9'
# teks = list_address[143]

# 1 - 50 = Jembrana
# 51 - 183 = Tabanan
# 184 - 246 = Badung
# 246 - 315 = Gianyar
# 316 - 374 = Klungkung
# 375 - 446 = Bangli
# 447 - 489 = Denpasar
# 490 - 567 = Karangasem
# 568 - 715 = Buleleng
#
for i in range(650,715):
    no_desa = i
    print(f'=========no desa : {no_desa}===========')
    time.sleep(10)
    teks = f'https://www.airbnb.co.id/s/{df_bali["desa"][no_desa]}--{df_bali["kecamatan"][no_desa]}--{df_bali["kabupaten"][no_desa]}/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&date_picker_type=calendar&l2_property_type_ids%5B%5D=4&search_type=search_query&source=structured_search_input_header'
    # DRIVER_PATH = '/path/to/chromedriver'
    # driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    # driver.get(teks)

    teks_pat = f'{df_bali_r["desa"][no_desa]}--{df_bali_r["kecamatan"][no_desa]}--{df_bali_r["kabupaten"][no_desa]}'
    print(teks_pat)

    response = requests.get(teks)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Cari elemen yang mengandung informasi halaman terakhir
    try:
        last_page = soup.find_all("a", class_="_833p2h")
        for link in last_page:
            link = link.text
        # Ekstrak nilai dari elemen tersebut
        num_page = int(link)
    except ValueError:
        num_page = 1
    # print(num_page)

# =================================================================================================

    df = pd.DataFrame (columns = ['link'])
    total_page = num_page
    # print(total_page)
    # pattern1 = r'https://airbnb\.co.id/s/Dangin\-Tukadaya\-\-Jembrana\-\-Jembrana/homes\?tab'
    pattern1 = r'https://airbnb\.co.id/s/'+teks_pat+r'/homes\?tab'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

    for i in range(total_page):
        print(f'index : {i}')
        if i == 0:
            teks = teks
        else:
            for index, item in enumerate(list_address):
                match = re.search(r'https://airbnb\.co\.id/help/home\?from=footer', item)
                if match:
                    teks = list_address[index-1]
        time.sleep(2)


        url = teks
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(2)
        # get title
        print(soup.title)
        all_links = soup.find_all("a")
        list_address = list()
        for link in all_links:
            link = link.get("href")
            list_address.append('https://airbnb.co.id'+link)

        for index, item in enumerate(list_address):
            match = re.search(pattern1, item)
            if match:
                find_stop = index


        list_item = list()
        for item in list(set(list_address[12:find_stop])):
            list_item.append(item)
            
        print(f'total link:{len(list_item)}')

        df2 = pd.DataFrame(list_item, columns = ['link'])
        df = pd.concat([df,df2], ignore_index=True)
    
    df_all = pd.concat([df_all,df], ignore_index=True)
    df_all

=========no desa : 650===========
Kalibukbuk--Buleleng--Buleleng
index : 0
<title>Resor di Anturan</title>
total link:22
index : 1
<title>Resor di Anturan</title>
total link:22
index : 2
<title>Resor di Anturan</title>
total link:22
index : 3
<title>Resor di Anturan</title>
total link:22
index : 4
<title>Resor di Anturan</title>
total link:22
index : 5
<title>Resor di Anturan</title>
total link:22
index : 6
<title>Resor di Anturan</title>
total link:22
index : 7
<title>Resor di Anturan</title>
total link:22
index : 8
<title>Resor di Anturan</title>
total link:22
index : 9
<title>Resor di Anturan</title>
total link:22
index : 10
<title>Resor di Anturan</title>
total link:22
index : 11
<title>Resor di Anturan</title>
total link:22
index : 12
<title>Resor di Anturan</title>
total link:22
index : 13
<title>Resor di Anturan</title>
total link:22
index : 14
<title>Resor di Anturan</title>
total link:22
=========no desa : 651===========
Anturan--Buleleng--Buleleng
index : 0
<title>Resor di An

In [7]:
df_all

,link
0,https://airbnb.co.id/rooms/609162905644832239?...
1,https://airbnb.co.id/s/Kalibukbuk--Buleleng--B...
2,https://airbnb.co.id/rooms/24537775?adults=1&c...
3,https://airbnb.co.id/rooms/704755467458104582?...
4,https://airbnb.co.id/rooms/646082416225353208?...
...,...
21445,https://airbnb.co.id/s/Sambirenteng--Tejakula-...
21446,https://airbnb.co.id/rooms/41757284?adults=1&c...
21447,https://airbnb.co.id/rooms/38652161?adults=1&c...
21448,https://airbnb.co.id/rooms/29883774?adults=1&c...


In [8]:
# df_all.to_excel('mining/jembrana-negara-16-Januari.xlsx', index=False)
df_all.to_excel('mining/buleleng-25-Januari-2.xlsx', index=False)

### Debug